# Table 11 and CSV Legal Representation National

## Contents
#### Setup
1. [import_packages](#import_packages) 

#### Stage 1 - [CSV Legal Representation National](#CSV_Legal_Representation_National)
2. [divorce_csv_legrep_table](#divorce_csv_legrep_table) - imports Divorce CSV legal representation data from S3 bucket into a temporary table
3. [fr_csv_legrep_table](#fr_csv_legrep_table) - imports Financial Remedy CSV legal representation data from S3 bucket into a temporary table
4. [csv_legrep_temp](#csv_legrep_temp) -  joins all the csv legal representation outputs from each case type (Adoption, Domestic Violence, Divorce, Financial Remedy, Private Law, and Public Law)

#### Stage 2 - [Table 11 Legal Representation](#Table_11_Legal_Representation)
5. [divorce_t11_legrep_table](#divorce_t11_legrep_table) -  imports Divorce Table 11 legal representation data from S3 bucket into a temporary table
6. [fin_rem_t11_legrep_table](#fin_rem_t11_legrep_table) -  imports Financial Remedy Table 11 legal representation data from S3 bucket into a temporary table
7. [t11_legrep_temp](#t11_legrep_temp) - joins all the Table 11 legal representation outputs from each case type (Adoption, Domestic Violence, Divorce, Financial Remedy, Private Law, and Public Law)

## 1. Import packages and set options 
<a name="import_packages"></a>

In [ ]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts
import numpy as np
import re

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

## Stage 1 - CSV Legal Representation National
<a name="CSV_Legal_Representation_National"></a>

### Import Divorce CSV Legal Representation
<a name="divorce_csv_legrep_table"></a>

#### Create the divorce_csv_legrep table

In [ ]:
#imports Divorce CSV Legal Representation data from S3 bucket into a temporary table
divorce_csv_legrep_table = pd.read_csv("s3://alpha-family-data/CSVs/Legal Representation/DIV_LEG_REP_CSV.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(divorce_csv_legrep_table, "divorce_csv_legrep")

##### divorce_csv_legrep validation

In [ ]:
#divorce_csv_legrep_count = pydb.read_sql_query("SELECT * from __temp__.divorce_csv_legrep limit 10")
#divorce_csv_legrep_count

### Import Financial Remedy CSV Legal Representation
<a name="fr_csv_legrep_table"></a>

#### Create the fr_csv_legrep table

In [ ]:
#imports Financial Remedy CSV Legal Representation data from S3 bucket into a temporary table
fr_csv_legrep_table = pd.read_csv("s3://alpha-family-data/CSVs/Legal Representation/FR_LEG_REP_CSV.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(fr_csv_legrep_table, "fr_csv_legrep")

##### fr_csv_legrep validation

In [ ]:
#fr_csv_legrep_count = pydb.read_sql_query("SELECT * from __temp__.fr_csv_legrep limit 10")
#fr_csv_legrep_count

### Final Output CSV Legal Representation
<a name="csv_legrep_temp"></a>

In [ ]:
pydb.create_temp_table(
f""" 
SELECT * FROM fcsq.adoption_legrep

UNION

SELECT * FROM __temp__.divorce_csv_legrep

UNION

SELECT * FROM fcsq.domestic_violence_legrep

UNION

SELECT * FROM __temp__.fr_csv_legrep

UNION 

SELECT * FROM fcsq.ca_leg_rep_csv
""",

"csv_legrep_temp")

In [ ]:
#csv_legrep_temp = pydb.read_sql_query("SELECT * FROM __temp__.csv_legrep_temp;")
#csv_legrep_temp

In [ ]:
#Orders the data by year, country, and county_ua
csv_legrep = pydb.read_sql_query("""
SELECT *
from __temp__.csv_legrep_temp
ORDER BY case_type,
category,
party,
gender,
representation,
year, 
quarter
""")

In [ ]:
#Export the final csv
csv_legrep.to_csv("s3://alpha-family-data/CSVs/Legal Representation/CSV Legal Representation National.csv", index = False)

## Stage 2 - Table 11 Legal Representation
<a name="Table_11_Legal_Representation"></a>

### Import Divorce Table 11 Legal Representation
<a name="divorce_t11_legrep_table"></a>

#### Create the divorce_t11_legrep table

In [ ]:
#imports Divorce Table 11 Legal Representation data from S3 bucket into a temporary table
divorce_t11_legrep_table = pd.read_csv("s3://alpha-family-data/CSVs/Legal Representation/DIV_TABLE_11_LOOKUP.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(divorce_t11_legrep_table, "div_t11_legrep")

##### divorce_t11_legrep validation

In [ ]:
#div_t11_legrep_count = pydb.read_sql_query("SELECT * from __temp__.div_t11_legrep limit 10")
#div_t11_legrep_count

### Import Financial Remedy Table 11 Legal Representation
<a name="fin_rem_t11_legrep_table"></a>

#### Create the fr_t11_legrep table

In [ ]:
#imports Financial Remedy Table 11 Legal Representation data from S3 bucket into a temporary table
fin_rem_t11_legrep_table = pd.read_csv("s3://alpha-family-data/CSVs/Legal Representation/FR_TABLE_11_LOOKUP.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(fin_rem_t11_legrep_table, "fin_rem_t11_legrep")

##### fr_t11_legrep validation

In [ ]:
#fin_rem_t11_legrep_count = pydb.read_sql_query("SELECT * from __temp__.fin_rem_t11_legrep limit 10")
#fin_rem_t11_legrep_count

### Final Output Table 11 Legal Representation
<a name="t11_legrep_temp"></a>

In [ ]:
pydb.create_temp_table(
f""" 
SELECT * FROM fcsq.adopt_legrep_t11_lookup

UNION 

SELECT * FROM fcsq.domestic_violence_table11_lookup

UNION

SELECT * FROM __temp__.div_t11_legrep

UNION 

SELECT * FROM __temp__.fin_rem_t11_legrep

UNION 

SELECT * FROM fcsq.ca_leg_rep_lookup

""",

"t11_legrep_temp")

In [ ]:
#t11_legrep_temp = pydb.read_sql_query("SELECT * FROM __temp__.t11_legrep_temp;")
#t11_legrep_temp

In [ ]:
#Orders the data by year, country, and county_ua
t11_legrep = pydb.read_sql_query("""
SELECT *
from __temp__.t11_legrep_temp
ORDER BY lookup
""")

In [ ]:
#Export the final csv
t11_legrep.to_csv("s3://alpha-family-data/CSVs/Legal Representation/Table_11_lookup.csv", index = False)